# Geoinformatics Project
---
# LEO satellites orbits simulator
###### Angelica Iseni, Emma Lodetti

The **goal** of this project is to compute the position of each LEO (Low Earth orbit) satellite in the constellation for every second of the day.

In this script you will be asked to enter: 
- The **number of orbital planes** of the costellation
- The **number of satellites** per orbital plane
- The **inclination** of the orbital planes with respect to the reference equatorial plane

This information will be used to create almanacs of the constellation satellites.

### Import libraries

In [5]:
# Useful libraries

import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink
import os
import subprocess
import pandas as pd
import pyproj
import plotly.graph_objects as go
import tkinter as tk
import oct2py
from oct2py import octave
from oct2py import Oct2Py

### Functions definition

In [2]:
# input parameters function
def user_input(var):
    
    n_orbits = num_orbits.value
    n_satellites = num_satellites.value
    inclination = orbit_inclination.value
    
    if n_orbits < 0 or n_satellites < 0 or inclination < 0:
        print('Error: The numbers cannot be negative. Please insert valid numbers.')
    else:
        print(f'You have selected {n_orbits} orbital planes with {n_satellites} satellites each, with an inclination of {inclination} degrees.')

name_list = []
        
# txt creation function
def create_satellite_txt(i_orbit, i_satellite):
    
    # Satellite name creation as "LEO XXYY", where XX = orbit number and YY = satellite number
    satellite_name = f'LEO{i_orbit:02}{i_satellite:02}'
    name_list.append(satellite_name)
    
    # M0 computation
    M0 = 360 / num_satellites * (i_satellite - 1) + 360 / num_satellites * ((i_orbit - 1) / num_orbits)
    
    # Omega0 computation
    Omega0 = (i_orbit - 1) * 180 / num_orbits
    
    # Almanac content
    output_content = f'OrbitRadius {orbit_radius}\nOrbitInclination {inclination}\nM0 {M0}\nOmega0 {Omega0}'
    
    # txt file path
    file_path = f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}/{satellite_name}.txt'
    
    # Write content in txt file
    with open(file_path, 'w') as file:
        file.write(output_content)
        
    # Return list of satellite nemes for plot selection
    return name_list

# Path selector
def get_path():
    global selected_path
    path = path_entry.get()
    path = path.replace('\\', '/')
    path = path.strip('"')
    selected_path = path
    print(f"Selected path: {path}")
    root.destroy() 
    return path


### Input parameters
Insert number of orbits, number of satellites per orbit and inclination of the orbit.

In [ ]:
# Widget creation
style = {'description_width': 'initial'}
num_orbits = widgets.IntText(description='Number of orbital planes:', value=0, style=style)
num_satellites = widgets.IntText(description='Number of satellites per orbital plane:', value=0, style=style)
orbit_inclination = widgets.FloatSlider(description='Orbit inclination:', min=0, max=90, step=1, style=style)
    
# Submit button
submit_button = widgets.Button(description='Submit')
submit_button.on_click(user_input)

display(num_orbits, num_satellites, orbit_inclination, submit_button)

### Folders and Almanacs creation as txt files
This tool produces two different folders: **AlmanacsXXYYZZ** and **SatellitePositionsXXYYZZ**, where:
- **XX** = number of orbital planes
- **YY** = number of satellites per orbital plane
- **ZZ** = inclination of orbital planes

**AlmanacsXXYYZZ** contains a set of txt files, one for each satellite of the constellation. Each txt file contains:
- Satellite name: **LEOXXYY**, where XX = orbit number and YY = satellite number
- Orbit radius = 7180 km
- Orbit inclination
- $M_0$ = mean anomaly
- $\Omega_0$ = right ascension of the ascending node

**SatellitePositionsXXYYZZ** is an empty folder that will contains a set of txt files computed by MATLAB. Each txt file, named as the corrensponding satellite, will contain its position in geodetic coordinates in each second of a day.

In [ ]:
# Convert values to integers
num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)

# Almanac folder creation
# "AlmanacsXXYYZZ": XX = number of orbital planes, YY = number of satellites per orbital plane, ZZ = inclination of orbital planes
if not os.path.exists(f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}'):
    os.makedirs(f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}')

# Satellite positon output folder creation
# "SatellitePositionsXXYYZZ": XX = number of orbital planes, YY = number of satellites per orbital plane, ZZ = inclination of orbital planes
if not os.path.exists(f'SatellitePositions{num_orbits:02}{num_satellites:02}{inclination:02}'):
    os.makedirs(f'SatellitePositions{num_orbits:02}{num_satellites:02}{inclination:02}')
    out_folder_name = f'SatellitePositions{num_orbits:02}{num_satellites:02}{inclination:02}'

# Orbit radius definition
orbit_radius = 7180 #km

# for cycle on orbit number
for i_orb in range(1, num_orbits + 1):
    # for cycle on satellite number per orbit
    for i_sat in range(1, num_satellites + 1):
            sat_name_list = create_satellite_txt(i_orb, i_sat)
        
print(f'Almanacs created successfully!')

## Trying Matlab Connection


In [9]:
# Set the path to the Octave executable
os.environ['OCTAVE_EXECUTABLE'] = 'C:\Program Files\GNU Octave\Octave-9.1.0\mingw64\bin'


In [11]:
# Definisci il percorso completo dello script Octave
script_path = r'C:\Users\emmal\Documents\GitHub\SimuLEO\MATLAB\SimuLEO.m'

# Definisci il percorso delle funzioni MATLAB
matlab_functions_path = r'C:\Users\emmal\Documents\GitHub\SimuLEO\MATLAB'

# Esegui lo script Octave utilizzando subprocess con il percorso delle funzioni MATLAB
subprocess.run(['octave', '--path', matlab_functions_path, script_path])

CompletedProcess(args=['octave', '--path', 'C:\\Users\\emmal\\Documents\\GitHub\\SimuLEO\\MATLAB', 'C:\\Users\\emmal\\Documents\\GitHub\\SimuLEO\\MATLAB\\SimuLEO.m'], returncode=0)

In [8]:
# to add a folder use:
octave.addpath('C:/Users/emmal/Documents/GitHub/SimuLEO/MATLAB')   
# to run the .m file :
octave.run('SimuLEO.m')

Oct2PyError: Octave evaluation error:
error: run: function called with too many outputs
error: called from:
    run at line -1, column -1

# Plot Ground Tracks

In [ ]:
selected_path = ""

root = tk.Tk()
root.title("Path Selector")

path_label = tk.Label(root, text="Enter the path of the solution you want to plot:")
path_label.pack()

path_entry = tk.Entry(root, width=50)
path_entry.pack()

submit_button = tk.Button(root, text="Submit", command=get_path)
submit_button.pack()

root.mainloop()

In [ ]:
folder_path = selected_path

sat_names = []

for file in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, file)):
        file = file.strip('.txt')
        sat_names.append(file)

#print(sat_names)

In [ ]:
# Plot selection
style = {'description_width': 'initial'}
plot_sat = widgets.Dropdown(
            options = sat_names,
            value = sat_names[0],
            description='Select satellite:',
            disabled = False,
            style = style,
            )

time_span = widgets.FloatSlider(description='Select time span in hours:', min=1, max=24, step=1, style=style)

display(plot_sat, time_span)

In [ ]:
path = selected_path + '/' + plot_sat.value + '.txt'
data = pd.read_csv(path, sep='\t', header=None, index_col=False, names=['latitude', 'longitude', 'height'])
time_span_seconds = int(time_span.value) * 3600

In [ ]:
# FOR EXPERIMENTS - DELETE LATER

data = pd.read_csv(f'SatellitePositions050530\LEO0101.txt', sep='\t', header=None, index_col=False, names=['latitude', 'longitude', 'height'])

In [ ]:
data

In [ ]:
fig = go.Figure(data=go.Scattergeo(
    lat = data['latitude'][0:time_span_seconds],
    lon = data['longitude'][0:time_span_seconds],
    mode = 'lines',
    line = dict(width = 2, color = 'red'),
))

fig.update_layout(
    title_text = f'Groundtrack of satellite {plot_sat.value} in {time_span.value} hours',
    showlegend = False,
    geo = dict(
        resolution = 50,
        showland = True,
        showlakes = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        projection_type = "miller",
        coastlinewidth = 1,
        coastlinecolor = 'rgb(204, 204, 204)',
        lataxis = dict(
            range = [-90, 90],
            showgrid = True,
            dtick = 10
        ),
        lonaxis = dict(
            range = [-180, 180],
            showgrid = True,
            dtick = 20
        ),
    )
)

fig.show()

In [ ]:
fig = go.Figure(data=go.Scattergeo(
    lat = data['latitude'][0:time_span_seconds],
    lon = data['longitude'][0:time_span_seconds],
    mode = 'lines',
    line = dict(width = 2, color = 'red'),
))

fig.update_layout(
    title_text = f'Groundtrack of satellite {plot_sat.value} in {time_span.value} hours',
    showlegend = False,
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        oceancolor = 'rgb(255, 255, 255)',
        countrycolor = 'rgb(204, 204, 204)',
        projection_type = "miller",
        coastlinewidth = 1,
        coastlinecolor = 'rgb(204, 204, 204)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)

fig.show()

In [ ]:
scl = ['rgb(255,0,0)', 'rgb(255,127,0)', 'rgb(255,255,0)', \
    'rgb(127,255,0)', 'rgb(0,255,0)', 'rgb(0,255,127)', \
    'rgb(0,255,255)', 'rgb(0,127,255)', 'rgb(0,0,255)',\
    'rgb(127,0,255)', 'rgb(255,0,255)', 'rgb(255,0,127)']
n_colors = len(scl)

fig = go.Figure()

for i in range(len(sat_names)):
    path = selected_path + '/' + sat_names[i] + '.txt'
    data = pd.read_csv(path, sep=' ', header=None, names=['latitude', 'longitude', 'height'])
    fig.add_trace(go.Scattergeo(
        lat = data['latitude'][0:2*3600],
        lon = data['longitude'][0:2*3600],
        mode = 'lines',
        name = sat_names[i],
        line = dict(width = 2, color = scl[i % n_colors]
        )))
  
fig.update_layout(
    title_text = 'Groundtracks of all the satellites of the costellation in 2 hours time span',
    showlegend = True,
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        oceancolor = 'rgb(255, 255, 255)',
        countrycolor = 'rgb(204, 204, 204)',
        projection_type = "miller",
        coastlinewidth = 1,
        coastlinecolor = 'rgb(204, 204, 204)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)

fig.show()